In [12]:
import cv2
import numpy as np
import json
import tensorflow as tf
from ultralytics import YOLO
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os

# Load JSON data for scoring model
file_path = "Athletes/KeypointDetection/JsonScore/Discurweper.json"
with open(file_path, 'r') as f:
    data = json.load(f)

# Define the expected number of keypoints (e.g., 17 keypoints, each with x, y, and visibility)
expected_keypoints = 17
expected_size = expected_keypoints * 3  # Each keypoint has x, y, visibility (3 values)

# Extract features (keypoints) and labels (scores)
features = []
scores = []

for segment, details in data['segments'].items():
    for annotation in details['annotations']:
        keypoints = annotation['keypoints']  # Flatten keypoints
        normalized_keypoints = []

        # Normalize using bbox dimensions
        bbox = annotation['bbox']
        
        # Ensure that keypoints contain a valid number of elements (x, y, and visibility for each keypoint)
        if len(keypoints) % 3 == 0:  # Check that each keypoint has x, y, and visibility
            for i in range(0, len(keypoints), 3):
                x = keypoints[i] / bbox[2]  # Normalize by width
                y = keypoints[i + 1] / bbox[3]  # Normalize by height
                visibility = keypoints[i + 2]  # Keep visibility as-is
                normalized_keypoints.extend([x, y, visibility])

            # Pad or trim the keypoints list to the expected size
            if len(normalized_keypoints) > expected_size:
                normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
            elif len(normalized_keypoints) < expected_size:
                normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

            features.append(normalized_keypoints)
            scores.append(details['score'])
        else:
            print(f"Skipping annotation with invalid keypoint length: {len(keypoints)}")

# Convert to NumPy arrays (after ensuring consistency)
X = np.array(features, dtype=np.float32)
y = np.array(scores, dtype=np.float32)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features using MinMaxScaler and save the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
scaler_path = "Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl"  # Correct path for saving the scaler
joblib.dump(scaler, scaler_path)  # Save the scaler
print(f"Scaler saved at {scaler_path}")

# Define a more complex model with added LSTM layers for sequential data
def create_model(input_dim):
    model = tf.keras.Sequential([
        # Reshape the input to be 3D for LSTM
        tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),  # Reshape to (1, features)
        
        # LSTM layer for sequential data (even though you have one timestep)
        tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
        tf.keras.layers.LSTM(64, activation='relu'),
        
        # Dense layers for feature learning
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


# Initialize and train the model with early stopping
model = create_model(X_train.shape[1])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# Calculate R² score
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2:.4f}")

# Save the model
model.save("Discurweper.h5")

# Load YOLO model for pose estimation
pose_model = YOLO("yolov8n-pose.pt")

# Path to the video
video_path = "Athletes/Testing/SegmentedVideosOriginal/segment_000959.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
frame_scores = []

# Load the saved scaler for normalization of keypoints
if os.path.exists(scaler_path):
    scaler_scoring = joblib.load(scaler_path)
else:
    # If the scaler doesn't exist, fit a new one (this case should not happen if you have already saved it)
    scaler_scoring = MinMaxScaler(feature_range=(0, 1))
    scaler_scoring.fit(X)  # Use `X` as the keypoints data
    joblib.dump(scaler_scoring, scaler_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Extract Keypoints
    results = pose_model(frame)
    if len(results) == 0 or results[0].keypoints is None:
        continue

    keypoints = results[0].keypoints.xy.cpu().numpy().flatten()  # x, y coordinates
    if len(keypoints) == 0:
        continue  # Skip frames without valid keypoints

    # Step 2: Normalize keypoints using bounding box dimensions
    bbox = results[0].boxes.xyxy.cpu().numpy()[0]  # x_min, y_min, x_max, y_max
    if bbox is None or len(bbox) != 4:
        continue  # Skip frames without valid bbox

    normalized_keypoints = []
    for i in range(0, len(keypoints), 2):  # Assuming keypoints are in x, y pairs
        x = keypoints[i] / bbox[2]  # Normalize by width
        y = keypoints[i + 1] / bbox[3]  # Normalize by height
        normalized_keypoints.extend([x, y])

    # Pad or trim the normalized keypoints to match the expected feature size (51 values)
    if len(normalized_keypoints) > expected_size:
        normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
    elif len(normalized_keypoints) < expected_size:
        normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

    # Step 3: Predict Score using the trained scoring model
    normalized_keypoints = np.array([normalized_keypoints], dtype=np.float32)

    # Normalize the features using the saved scaler
    normalized_keypoints = scaler_scoring.transform(normalized_keypoints)

    # Predict score from the model
    score = model.predict(normalized_keypoints).flatten()[0]

    # Clip the predicted score to ensure it's between 1 and 5
    score = np.clip(score, 1, 5)

    # Round the score to the nearest 0.5
    rounded_score = round(score * 2) / 2
    frame_scores.append(rounded_score)

    # Step 4: Annotate the frame with the score
    cv2.putText(frame, f"Score: {rounded_score:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show frame
    frame_count += 1
    cv2.imshow('Video', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Step 5: Aggregate Results
average_score = np.mean(frame_scores) if frame_scores else 0
print(f"Average Score for the Video: {average_score:.2f}")


Scaler saved at /Users/alessiacolumban/Desktop/TeamProject-GradindSysAthletes/Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl
Epoch 1/100


/opt/anaconda3/envs/athletes/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 4.2902 - mae: 1.7014 - val_loss: 6.5030 - val_mae: 2.3060
Epoch 2/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5024 - mae: 0.9931 - val_loss: 5.6391 - val_mae: 2.1221
Epoch 3/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4445 - mae: 0.9802 - val_loss: 4.5591 - val_mae: 1.8927
Epoch 4/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2945 - mae: 0.9359 - val_loss: 3.4288 - val_mae: 1.6424
Epoch 5/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2076 - mae: 0.9042 - val_loss: 3.2717 - val_mae: 1.6122
Epoch 6/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1885 - mae: 0.8724 - val_loss: 2.8943 - val_mae: 1.4925
Epoch 7/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1771 - mae: 0.8797 - val_loss: 2.3485 - val_mae: 1.3349
Epoch 8/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0965 - mae: 0.8495 - val_loss: 1.6419 - val_mae: 1.1202
Epoch 9/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0040 - mae: 0.8

R² Score: 0.4933

0: 384x640 2 persons, 47.4ms
Speed: 3.0ms preprocess, 47.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 2 persons, 74.7ms
Speed: 1.1ms preprocess, 74.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 2 persons, 42.2ms
Speed: 0.7ms preprocess, 42.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 2 persons, 44.3ms
Speed: 0.9ms preprocess, 44.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

0: 384x640 2 persons, 46.6ms
Speed: 0.8ms preprocess, 46.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 2 persons, 40.3ms
Speed: 0.7ms preprocess, 40.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14m

In [13]:
import cv2
import numpy as np
import json
import tensorflow as tf
from ultralytics import YOLO
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os

# Load JSON data for scoring model
file_path = "Athletes/KeypointDetection/JsonScore/Estafette.json"
with open(file_path, 'r') as f:
    data = json.load(f)

# Define the expected number of keypoints (e.g., 17 keypoints, each with x, y, and visibility)
expected_keypoints = 17
expected_size = expected_keypoints * 3  # Each keypoint has x, y, visibility (3 values)

# Extract features (keypoints) and labels (scores)
features = []
scores = []

for segment, details in data['segments'].items():
    for annotation in details['annotations']:
        keypoints = annotation['keypoints']  # Flatten keypoints
        normalized_keypoints = []

        # Normalize using bbox dimensions
        bbox = annotation['bbox']
        
        # Ensure that keypoints contain a valid number of elements (x, y, and visibility for each keypoint)
        if len(keypoints) % 3 == 0:  # Check that each keypoint has x, y, and visibility
            for i in range(0, len(keypoints), 3):
                x = keypoints[i] / bbox[2]  # Normalize by width
                y = keypoints[i + 1] / bbox[3]  # Normalize by height
                visibility = keypoints[i + 2]  # Keep visibility as-is
                normalized_keypoints.extend([x, y, visibility])

            # Pad or trim the keypoints list to the expected size
            if len(normalized_keypoints) > expected_size:
                normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
            elif len(normalized_keypoints) < expected_size:
                normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

            features.append(normalized_keypoints)
            scores.append(details['score'])
        else:
            print(f"Skipping annotation with invalid keypoint length: {len(keypoints)}")

# Convert to NumPy arrays (after ensuring consistency)
X = np.array(features, dtype=np.float32)
y = np.array(scores, dtype=np.float32)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features using MinMaxScaler and save the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
scaler_path = "thletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl"  # Correct path for saving the scaler
joblib.dump(scaler, scaler_path)  # Save the scaler
print(f"Scaler saved at {scaler_path}")

# Define a more complex model with added LSTM layers for sequential data
def create_model(input_dim):
    model = tf.keras.Sequential([
        # Reshape the input to be 3D for LSTM
        tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),  # Reshape to (1, features)
        
        # LSTM layer for sequential data (even though you have one timestep)
        tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
        tf.keras.layers.LSTM(64, activation='relu'),
        
        # Dense layers for feature learning
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


# Initialize and train the model with early stopping
model = create_model(X_train.shape[1])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# Calculate R² score
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2:.4f}")

# Save the model
model.save("Estafette.h5")

# Load YOLO model for pose estimation
pose_model = YOLO("yolov8n-pose.pt")

# Path to the video
video_path = "Athletes/Testing/Testing/SegmentedVideosOriginal/segment_005629.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
frame_scores = []

# Load the saved scaler for normalization of keypoints
if os.path.exists(scaler_path):
    scaler_scoring = joblib.load(scaler_path)
else:
    # If the scaler doesn't exist, fit a new one (this case should not happen if you have already saved it)
    scaler_scoring = MinMaxScaler(feature_range=(0, 1))
    scaler_scoring.fit(X)  # Use `X` as the keypoints data
    joblib.dump(scaler_scoring, scaler_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Extract Keypoints
    results = pose_model(frame)
    if len(results) == 0 or results[0].keypoints is None:
        continue

    keypoints = results[0].keypoints.xy.cpu().numpy().flatten()  # x, y coordinates
    if len(keypoints) == 0:
        continue  # Skip frames without valid keypoints

    # Step 2: Normalize keypoints using bounding box dimensions
    bbox = results[0].boxes.xyxy.cpu().numpy()[0]  # x_min, y_min, x_max, y_max
    if bbox is None or len(bbox) != 4:
        continue  # Skip frames without valid bbox

    normalized_keypoints = []
    for i in range(0, len(keypoints), 2):  # Assuming keypoints are in x, y pairs
        x = keypoints[i] / bbox[2]  # Normalize by width
        y = keypoints[i + 1] / bbox[3]  # Normalize by height
        normalized_keypoints.extend([x, y])

    # Pad or trim the normalized keypoints to match the expected feature size (51 values)
    if len(normalized_keypoints) > expected_size:
        normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
    elif len(normalized_keypoints) < expected_size:
        normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

    # Step 3: Predict Score using the trained scoring model
    normalized_keypoints = np.array([normalized_keypoints], dtype=np.float32)

    # Normalize the features using the saved scaler
    normalized_keypoints = scaler_scoring.transform(normalized_keypoints)

    # Predict score from the model
    score = model.predict(normalized_keypoints).flatten()[0]

    # Clip the predicted score to ensure it's between 1 and 5
    score = np.clip(score, 1, 5)

    # Round the score to the nearest 0.5
    rounded_score = round(score * 2) / 2
    frame_scores.append(rounded_score)

    # Step 4: Annotate the frame with the score
    cv2.putText(frame, f"Score: {rounded_score:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show frame
    frame_count += 1
    cv2.imshow('Video', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Step 5: Aggregate Results
average_score = np.mean(frame_scores) if frame_scores else 0
print(f"Average Score for the Video: {average_score:.2f}")


Scaler saved at /Users/alessiacolumban/Desktop/TeamProject-GradindSysAthletes/Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl
Epoch 1/100


/opt/anaconda3/envs/athletes/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 7.3046 - mae: 2.3435 - val_loss: 7.4237 - val_mae: 2.5625
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5218 - mae: 0.9636 - val_loss: 6.2886 - val_mae: 2.3809
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4680 - mae: 0.9435 - val_loss: 5.5703 - val_mae: 2.2567
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3663 - mae: 0.9068 - val_loss: 4.7188 - val_mae: 2.0858
Epoch 5/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2163 - mae: 0.8406 - val_loss: 3.7333 - val_mae: 1.8590
Epoch 6/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1204 - mae: 0.7891 - val_loss: 3.3852 - val_mae: 1.7673
Epoch 7/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9922 - mae: 0.7354 - val_loss: 2.5748 - val_mae: 1.5232
Epoch 8/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9841 - mae: 0.7320 - val_loss: 2.3139 - val_mae: 1.4288
Epoch 9/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9251 - mae: 0.6

R² Score: 0.5499

0: 384x640 2 persons, 65.3ms
Speed: 4.5ms preprocess, 65.3ms inference, 8.2ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

0: 384x640 2 persons, 57.4ms
Speed: 1.0ms preprocess, 57.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 2 persons, 46.8ms
Speed: 0.8ms preprocess, 46.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 2 persons, 40.2ms
Speed: 0.8ms preprocess, 40.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 2 persons, 40.7ms
Speed: 0.8ms preprocess, 40.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 2 persons, 46.5ms
Speed: 0.7ms preprocess, 46.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17m

In [18]:
import cv2
import numpy as np
import json
import tensorflow as tf
from ultralytics import YOLO
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os

# Load JSON data for scoring model
file_path = "Athletes/KeypointDetection/JsonScore/Hoogspringen.json"
with open(file_path, 'r') as f:
    data = json.load(f)

# Define the expected number of keypoints (e.g., 17 keypoints, each with x, y, and visibility)
expected_keypoints = 17
expected_size = expected_keypoints * 3  # Each keypoint has x, y, visibility (3 values)

# Extract features (keypoints) and labels (scores)
features = []
scores = []

for segment, details in data['segments'].items():
    for annotation in details['annotations']:
        keypoints = annotation['keypoints']  # Flatten keypoints
        normalized_keypoints = []

        # Normalize using bbox dimensions
        bbox = annotation['bbox']
        
        # Ensure that keypoints contain a valid number of elements (x, y, and visibility for each keypoint)
        if len(keypoints) % 3 == 0:  # Check that each keypoint has x, y, and visibility
            for i in range(0, len(keypoints), 3):
                x = keypoints[i] / bbox[2]  # Normalize by width
                y = keypoints[i + 1] / bbox[3]  # Normalize by height
                visibility = keypoints[i + 2]  # Keep visibility as-is
                normalized_keypoints.extend([x, y, visibility])

            # Pad or trim the keypoints list to the expected size
            if len(normalized_keypoints) > expected_size:
                normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
            elif len(normalized_keypoints) < expected_size:
                normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

            features.append(normalized_keypoints)
            scores.append(details['score'])
        else:
            print(f"Skipping annotation with invalid keypoint length: {len(keypoints)}")

# Convert to NumPy arrays (after ensuring consistency)
X = np.array(features, dtype=np.float32)
y = np.array(scores, dtype=np.float32)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features using MinMaxScaler and save the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
scaler_path = "Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl"  # Correct path for saving the scaler
joblib.dump(scaler, scaler_path)  # Save the scaler
print(f"Scaler saved at {scaler_path}")

# Define a more complex model with added LSTM layers for sequential data
def create_model(input_dim):
    model = tf.keras.Sequential([
        # Reshape the input to be 3D for LSTM
        tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),  # Reshape to (1, features)
        
        # LSTM layer for sequential data (even though you have one timestep)
        tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
        tf.keras.layers.LSTM(64, activation='relu'),
        
        # Dense layers for feature learning
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


# Initialize and train the model with early stopping
model = create_model(X_train.shape[1])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# Calculate R² score
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2:.4f}")

# Save the model
model.save("Hoogspringen.h5")

# Load YOLO model for pose estimation
pose_model = YOLO("yolov8n-pose.pt")

# Path to the video
video_path = "Athletes/Testing/Testing/SegmentedVideosOriginal/segment_003894.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
frame_scores = []

# Load the saved scaler for normalization of keypoints
if os.path.exists(scaler_path):
    scaler_scoring = joblib.load(scaler_path)
else:
    # If the scaler doesn't exist, fit a new one (this case should not happen if you have already saved it)
    scaler_scoring = MinMaxScaler(feature_range=(0, 1))
    scaler_scoring.fit(X)  # Use `X` as the keypoints data
    joblib.dump(scaler_scoring, scaler_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Extract Keypoints
    results = pose_model(frame)
    if len(results) == 0 or results[0].keypoints is None:
        continue

    keypoints = results[0].keypoints.xy.cpu().numpy().flatten()  # x, y coordinates
    if len(keypoints) == 0:
        continue  # Skip frames without valid keypoints

    # Step 2: Normalize keypoints using bounding box dimensions
    bbox = results[0].boxes.xyxy.cpu().numpy()[0]  # x_min, y_min, x_max, y_max
    if bbox is None or len(bbox) != 4:
        continue  # Skip frames without valid bbox

    normalized_keypoints = []
    for i in range(0, len(keypoints), 2):  # Assuming keypoints are in x, y pairs
        x = keypoints[i] / bbox[2]  # Normalize by width
        y = keypoints[i + 1] / bbox[3]  # Normalize by height
        normalized_keypoints.extend([x, y])

    # Pad or trim the normalized keypoints to match the expected feature size (51 values)
    if len(normalized_keypoints) > expected_size:
        normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
    elif len(normalized_keypoints) < expected_size:
        normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

    # Step 3: Predict Score using the trained scoring model
    normalized_keypoints = np.array([normalized_keypoints], dtype=np.float32)

    # Normalize the features using the saved scaler
    normalized_keypoints = scaler_scoring.transform(normalized_keypoints)

    # Predict score from the model
    score = model.predict(normalized_keypoints).flatten()[0]

    # Clip the predicted score to ensure it's between 1 and 5
    score = np.clip(score, 1, 5)

    # Round the score to the nearest 0.5
    rounded_score = round(score * 2) / 2
    frame_scores.append(rounded_score)

    # Step 4: Annotate the frame with the score
    cv2.putText(frame, f"Score: {rounded_score:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show frame
    frame_count += 1
    cv2.imshow('Video', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Step 5: Aggregate Results
average_score = np.mean(frame_scores) if frame_scores else 0
print(f"Average Score for the Video: {average_score:.2f}")


Scaler saved at /Users/alessiacolumban/Desktop/TeamProject-GradindSysAthletes/Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl
Epoch 1/100


/opt/anaconda3/envs/athletes/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 10.5497 - mae: 2.9112 - val_loss: 7.4646 - val_mae: 2.4271
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.3978 - mae: 1.2215 - val_loss: 6.2814 - val_mae: 2.2106
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.1305 - mae: 1.1572 - val_loss: 5.0465 - val_mae: 1.9395
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8164 - mae: 1.0480 - val_loss: 3.9396 - val_mae: 1.6791
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.6632 - mae: 1.0221 - val_loss: 3.0675 - val_mae: 1.4360
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4857 - mae: 0.9403 - val_loss: 2.6094 - val_mae: 1.2945
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4095 - mae: 0.9283 - val_loss: 2.0075 - val_mae: 1.1181
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4619 - mae: 0.9649 - val_loss: 1.6860 - val_mae: 1.0194
Epoch 9/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3884 - mae: 0.

R² Score: 0.4116

0: 384x640 9 persons, 47.9ms
Speed: 1.9ms preprocess, 47.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 10 persons, 50.2ms
Speed: 1.3ms preprocess, 50.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 7 persons, 38.9ms
Speed: 0.7ms preprocess, 38.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 7 persons, 42.5ms
Speed: 0.7ms preprocess, 42.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

0: 384x640 7 persons, 43.2ms
Speed: 0.7ms preprocess, 43.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

0: 384x640 7 persons, 64.5ms
Speed: 2.3ms preprocess, 64.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33

In [19]:
import cv2
import numpy as np
import json
import tensorflow as tf
from ultralytics import YOLO
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os

# Load JSON data for scoring model
file_path = "Athletes/KeypointDetection/JsonScore/Hordenlopen.json"
with open(file_path, 'r') as f:
    data = json.load(f)

# Define the expected number of keypoints (e.g., 17 keypoints, each with x, y, and visibility)
expected_keypoints = 17
expected_size = expected_keypoints * 3  # Each keypoint has x, y, visibility (3 values)

# Extract features (keypoints) and labels (scores)
features = []
scores = []

for segment, details in data['segments'].items():
    for annotation in details['annotations']:
        keypoints = annotation['keypoints']  # Flatten keypoints
        normalized_keypoints = []

        # Normalize using bbox dimensions
        bbox = annotation['bbox']
        
        # Ensure that keypoints contain a valid number of elements (x, y, and visibility for each keypoint)
        if len(keypoints) % 3 == 0:  # Check that each keypoint has x, y, and visibility
            for i in range(0, len(keypoints), 3):
                x = keypoints[i] / bbox[2]  # Normalize by width
                y = keypoints[i + 1] / bbox[3]  # Normalize by height
                visibility = keypoints[i + 2]  # Keep visibility as-is
                normalized_keypoints.extend([x, y, visibility])

            # Pad or trim the keypoints list to the expected size
            if len(normalized_keypoints) > expected_size:
                normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
            elif len(normalized_keypoints) < expected_size:
                normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

            features.append(normalized_keypoints)
            scores.append(details['score'])
        else:
            print(f"Skipping annotation with invalid keypoint length: {len(keypoints)}")

# Convert to NumPy arrays (after ensuring consistency)
X = np.array(features, dtype=np.float32)
y = np.array(scores, dtype=np.float32)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features using MinMaxScaler and save the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
scaler_path = "Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl"  # Correct path for saving the scaler
joblib.dump(scaler, scaler_path)  # Save the scaler
print(f"Scaler saved at {scaler_path}")

# Define a more complex model with added LSTM layers for sequential data
def create_model(input_dim):
    model = tf.keras.Sequential([
        # Reshape the input to be 3D for LSTM
        tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),  # Reshape to (1, features)
        
        # LSTM layer for sequential data (even though you have one timestep)
        tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
        tf.keras.layers.LSTM(64, activation='relu'),
        
        # Dense layers for feature learning
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


# Initialize and train the model with early stopping
model = create_model(X_train.shape[1])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# Calculate R² score
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2:.4f}")

# Save the model
model.save("Hordelopen.h5")

# Load YOLO model for pose estimation
pose_model = YOLO("yolov8n-pose.pt")

# Path to the video
video_path = "Athletes/Testing/Testing/SegmentedVideosOriginal/segment_007294.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
frame_scores = []

# Load the saved scaler for normalization of keypoints
if os.path.exists(scaler_path):
    scaler_scoring = joblib.load(scaler_path)
else:
    # If the scaler doesn't exist, fit a new one (this case should not happen if you have already saved it)
    scaler_scoring = MinMaxScaler(feature_range=(0, 1))
    scaler_scoring.fit(X)  # Use `X` as the keypoints data
    joblib.dump(scaler_scoring, scaler_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Extract Keypoints
    results = pose_model(frame)
    if len(results) == 0 or results[0].keypoints is None:
        continue

    keypoints = results[0].keypoints.xy.cpu().numpy().flatten()  # x, y coordinates
    if len(keypoints) == 0:
        continue  # Skip frames without valid keypoints

    # Step 2: Normalize keypoints using bounding box dimensions
    bbox = results[0].boxes.xyxy.cpu().numpy()[0]  # x_min, y_min, x_max, y_max
    if bbox is None or len(bbox) != 4:
        continue  # Skip frames without valid bbox

    normalized_keypoints = []
    for i in range(0, len(keypoints), 2):  # Assuming keypoints are in x, y pairs
        x = keypoints[i] / bbox[2]  # Normalize by width
        y = keypoints[i + 1] / bbox[3]  # Normalize by height
        normalized_keypoints.extend([x, y])

    # Pad or trim the normalized keypoints to match the expected feature size (51 values)
    if len(normalized_keypoints) > expected_size:
        normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
    elif len(normalized_keypoints) < expected_size:
        normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

    # Step 3: Predict Score using the trained scoring model
    normalized_keypoints = np.array([normalized_keypoints], dtype=np.float32)

    # Normalize the features using the saved scaler
    normalized_keypoints = scaler_scoring.transform(normalized_keypoints)

    # Predict score from the model
    score = model.predict(normalized_keypoints).flatten()[0]

    # Clip the predicted score to ensure it's between 1 and 5
    score = np.clip(score, 1, 5)

    # Round the score to the nearest 0.5
    rounded_score = round(score * 2) / 2
    frame_scores.append(rounded_score)

    # Step 4: Annotate the frame with the score
    cv2.putText(frame, f"Score: {rounded_score:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show frame
    frame_count += 1
    cv2.imshow('Video', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Step 5: Aggregate Results
average_score = np.mean(frame_scores) if frame_scores else 0
print(f"Average Score for the Video: {average_score:.2f}")


Scaler saved at /Users/alessiacolumban/Desktop/TeamProject-GradindSysAthletes/Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl


/opt/anaconda3/envs/athletes/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 6.0053 - mae: 2.0456 - val_loss: 10.6743 - val_mae: 3.0636
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.6134 - mae: 0.9944 - val_loss: 8.3459 - val_mae: 2.6917
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5639 - mae: 0.9890 - val_loss: 6.5045 - val_mae: 2.3421
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3623 - mae: 0.9042 - val_loss: 4.6659 - val_mae: 1.9130
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3889 - mae: 0.9139 - val_loss: 3.9578 - val_mae: 1.7572
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2262 - mae: 0.8454 - val_loss: 2.4689 - val_mae: 1.3014
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1698 - mae: 0.8392 - val_loss: 2.3021 - val_mae: 1.2543
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2212 - mae: 0.8459 - val_loss: 1.4487 - val_mae: 0.9846
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.00

R² Score: 0.4667

0: 384x640 1 person, 58.7ms
Speed: 2.2ms preprocess, 58.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step

0: 384x640 1 person, 70.3ms
Speed: 1.5ms preprocess, 70.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 1 person, 43.9ms
Speed: 0.7ms preprocess, 43.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 1 person, 38.4ms
Speed: 0.9ms preprocess, 38.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 1 person, 38.2ms
Speed: 1.0ms preprocess, 38.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 1 person, 40.9ms
Speed: 0.8ms preprocess, 40.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/ste

In [26]:
import cv2
import numpy as np
import json
import tensorflow as tf
from ultralytics import YOLO
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os

# Load JSON data for scoring model
file_path = "Athletes/KeypointDetection/JsonScore/Kogelstoten.json"
with open(file_path, 'r') as f:
    data = json.load(f)

# Define the expected number of keypoints (e.g., 17 keypoints, each with x, y, and visibility)
expected_keypoints = 17
expected_size = expected_keypoints * 3  # Each keypoint has x, y, visibility (3 values)

# Extract features (keypoints) and labels (scores)
features = []
scores = []

for segment, details in data['segments'].items():
    for annotation in details['annotations']:
        keypoints = annotation['keypoints']  # Flatten keypoints
        normalized_keypoints = []

        # Normalize using bbox dimensions
        bbox = annotation['bbox']
        
        # Ensure that keypoints contain a valid number of elements (x, y, and visibility for each keypoint)
        if len(keypoints) % 3 == 0:  # Check that each keypoint has x, y, and visibility
            for i in range(0, len(keypoints), 3):
                x = keypoints[i] / bbox[2]  # Normalize by width
                y = keypoints[i + 1] / bbox[3]  # Normalize by height
                visibility = keypoints[i + 2]  # Keep visibility as-is
                normalized_keypoints.extend([x, y, visibility])

            # Pad or trim the keypoints list to the expected size
            if len(normalized_keypoints) > expected_size:
                normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
            elif len(normalized_keypoints) < expected_size:
                normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

            features.append(normalized_keypoints)
            scores.append(details['score'])
        else:
            print(f"Skipping annotation with invalid keypoint length: {len(keypoints)}")

# Convert to NumPy arrays (after ensuring consistency)
X = np.array(features, dtype=np.float32)
y = np.array(scores, dtype=np.float32)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features using MinMaxScaler and save the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
scaler_path = "Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl"  # Correct path for saving the scaler
joblib.dump(scaler, scaler_path)  # Save the scaler
print(f"Scaler saved at {scaler_path}")

# Define a more complex model with added LSTM layers for sequential data
def create_model(input_dim):
    model = tf.keras.Sequential([
        # Reshape the input to be 3D for LSTM
        tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),  # Reshape to (1, features)
        
        # LSTM layer for sequential data (even though you have one timestep)
        tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
        tf.keras.layers.LSTM(64, activation='relu'),
        
        # Dense layers for feature learning
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


# Initialize and train the model with early stopping
model = create_model(X_train.shape[1])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# Calculate R² score
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2:.4f}")

# Save the model
model.save("Kogelstoten.h5")

# Load YOLO model for pose estimation
pose_model = YOLO("yolov8n-pose.pt")

# Path to the video
video_path = "Athletes/Testing/Testing/SegmentedVideosOriginal/segment_003023.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
frame_scores = []

# Load the saved scaler for normalization of keypoints
if os.path.exists(scaler_path):
    scaler_scoring = joblib.load(scaler_path)
else:
    # If the scaler doesn't exist, fit a new one (this case should not happen if you have already saved it)
    scaler_scoring = MinMaxScaler(feature_range=(0, 1))
    scaler_scoring.fit(X)  # Use `X` as the keypoints data
    joblib.dump(scaler_scoring, scaler_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Extract Keypoints
    results = pose_model(frame)
    if len(results) == 0 or results[0].keypoints is None:
        continue

    keypoints = results[0].keypoints.xy.cpu().numpy().flatten()  # x, y coordinates
    if len(keypoints) == 0:
        continue  # Skip frames without valid keypoints

    # Step 2: Normalize keypoints using bounding box dimensions
    bbox = results[0].boxes.xyxy.cpu().numpy()[0]  # x_min, y_min, x_max, y_max
    if bbox is None or len(bbox) != 4:
        continue  # Skip frames without valid bbox

    normalized_keypoints = []
    for i in range(0, len(keypoints), 2):  # Assuming keypoints are in x, y pairs
        x = keypoints[i] / bbox[2]  # Normalize by width
        y = keypoints[i + 1] / bbox[3]  # Normalize by height
        normalized_keypoints.extend([x, y])

    # Pad or trim the normalized keypoints to match the expected feature size (51 values)
    if len(normalized_keypoints) > expected_size:
        normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
    elif len(normalized_keypoints) < expected_size:
        normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

    # Step 3: Predict Score using the trained scoring model
    normalized_keypoints = np.array([normalized_keypoints], dtype=np.float32)

    # Normalize the features using the saved scaler
    normalized_keypoints = scaler_scoring.transform(normalized_keypoints)

    # Predict score from the model
    score = model.predict(normalized_keypoints).flatten()[0]

    # Clip the predicted score to ensure it's between 1 and 5
    score = np.clip(score, 1, 5)

    # Round the score to the nearest 0.5
    rounded_score = round(score * 2) / 2
    frame_scores.append(rounded_score)

    # Step 4: Annotate the frame with the score
    cv2.putText(frame, f"Score: {rounded_score:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show frame
    frame_count += 1
    cv2.imshow('Video', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Step 5: Aggregate Results
average_score = np.mean(frame_scores) if frame_scores else 0
print(f"Average Score for the Video: {average_score:.2f}")


Scaler saved at /Users/alessiacolumban/Desktop/TeamProject-GradindSysAthletes/Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl
Epoch 1/100


/opt/anaconda3/envs/athletes/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 8.7938 - mae: 2.6457 - val_loss: 8.2016 - val_mae: 2.7047
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5339 - mae: 0.9746 - val_loss: 6.7572 - val_mae: 2.4328
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0664 - mae: 0.8240 - val_loss: 5.6184 - val_mae: 2.2056
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8717 - mae: 0.7203 - val_loss: 4.6085 - val_mae: 1.9891
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7982 - mae: 0.7045 - val_loss: 3.5625 - val_mae: 1.7446
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7438 - mae: 0.6685 - val_loss: 2.9133 - val_mae: 1.5609
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5977 - mae: 0.5904 - val_loss: 2.5087 - val_mae: 1.4511
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5620 - mae: 0.5800 - val_loss: 1.9325 - val_mae: 1.2674
Epoch 9/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5458 - mae: 0.5

R² Score: 0.7468

0: 384x640 2 persons, 51.7ms
Speed: 1.2ms preprocess, 51.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 3 persons, 49.9ms
Speed: 0.9ms preprocess, 49.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 3 persons, 41.8ms
Speed: 0.7ms preprocess, 41.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 3 persons, 46.1ms
Speed: 0.6ms preprocess, 46.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

0: 384x640 3 persons, 52.5ms
Speed: 0.7ms preprocess, 52.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

0: 384x640 3 persons, 84.9ms
Speed: 1.4ms preprocess, 84.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15m

In [5]:
import cv2
import numpy as np
import json
import tensorflow as tf
from ultralytics import YOLO
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os

# Load JSON data for scoring model
file_path = "Athletes/KeypointDetection/JsonScore/Speerwerpen.json"
with open(file_path, 'r') as f:
    data = json.load(f)

# Define the expected number of keypoints (e.g., 17 keypoints, each with x, y, and visibility)
expected_keypoints = 17
expected_size = expected_keypoints * 3  # Each keypoint has x, y, visibility (3 values)

# Extract features (keypoints) and labels (scores)
features = []
scores = []

for segment, details in data['segments'].items():
    for annotation in details['annotations']:
        keypoints = annotation['keypoints']  # Flatten keypoints
        normalized_keypoints = []

        # Normalize using bbox dimensions
        bbox = annotation['bbox']
        
        # Ensure that keypoints contain a valid number of elements (x, y, and visibility for each keypoint)
        if len(keypoints) % 3 == 0:  # Check that each keypoint has x, y, and visibility
            for i in range(0, len(keypoints), 3):
                x = keypoints[i] / bbox[2]  # Normalize by width
                y = keypoints[i + 1] / bbox[3]  # Normalize by height
                visibility = keypoints[i + 2]  # Keep visibility as-is
                normalized_keypoints.extend([x, y, visibility])

            # Pad or trim the keypoints list to the expected size
            if len(normalized_keypoints) > expected_size:
                normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
            elif len(normalized_keypoints) < expected_size:
                normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

            features.append(normalized_keypoints)
            scores.append(details['score'])
        else:
            print(f"Skipping annotation with invalid keypoint length: {len(keypoints)}")

# Convert to NumPy arrays (after ensuring consistency)
X = np.array(features, dtype=np.float32)
y = np.array(scores, dtype=np.float32)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features using MinMaxScaler and save the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
scaler_path = "Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl"  # Correct path for saving the scaler
joblib.dump(scaler, scaler_path)  # Save the scaler
print(f"Scaler saved at {scaler_path}")

# Define a more complex model with added LSTM layers for sequential data
def create_model(input_dim):
    model = tf.keras.Sequential([
        # Reshape input for sequential data
        tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),

        # LSTM layers
        tf.keras.layers.LSTM(128, activation='tanh', return_sequences=True, recurrent_dropout=0.2),
        tf.keras.layers.LSTM(64, activation='tanh', recurrent_dropout=0.2),
        tf.keras.layers.Dropout(0.2),

        # Dense layers with Batch Normalization and Dropout
        tf.keras.layers.Dense(128),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(64, activation='relu'),

        # Output layer
        tf.keras.layers.Dense(1, activation='linear')  # For regression
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])
    return model



# Initialize and train the model with early stopping
model = create_model(X_train.shape[1])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# Calculate R² score
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2:.4f}")

# Save the model
model.save("Speerwerpen.h5")

# Load YOLO model for pose estimation
pose_model = YOLO("yolov8n-pose.pt")

# Path to the video
video_path = "Athletes/Testing/Testing/SegmentedVideosOriginal/segment_006403.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
frame_scores = []

# Load the saved scaler for normalization of keypoints
if os.path.exists(scaler_path):
    scaler_scoring = joblib.load(scaler_path)
else:
    # If the scaler doesn't exist, fit a new one (this case should not happen if you have already saved it)
    scaler_scoring = MinMaxScaler(feature_range=(0, 1))
    scaler_scoring.fit(X)  # Use `X` as the keypoints data
    joblib.dump(scaler_scoring, scaler_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Extract Keypoints
    results = pose_model(frame)
    if len(results) == 0 or results[0].keypoints is None:
        continue

    keypoints = results[0].keypoints.xy.cpu().numpy().flatten()  # x, y coordinates
    if len(keypoints) == 0:
        continue  # Skip frames without valid keypoints

    # Step 2: Normalize keypoints using bounding box dimensions
    bbox = results[0].boxes.xyxy.cpu().numpy()[0]  # x_min, y_min, x_max, y_max
    if bbox is None or len(bbox) != 4:
        continue  # Skip frames without valid bbox

    normalized_keypoints = []
    for i in range(0, len(keypoints), 2):  # Assuming keypoints are in x, y pairs
        x = keypoints[i] / bbox[2]  # Normalize by width
        y = keypoints[i + 1] / bbox[3]  # Normalize by height
        normalized_keypoints.extend([x, y])

    # Pad or trim the normalized keypoints to match the expected feature size (51 values)
    if len(normalized_keypoints) > expected_size:
        normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
    elif len(normalized_keypoints) < expected_size:
        normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

    # Step 3: Predict Score using the trained scoring model
    normalized_keypoints = np.array([normalized_keypoints], dtype=np.float32)

    # Normalize the features using the saved scaler
    normalized_keypoints = scaler_scoring.transform(normalized_keypoints)

    # Predict score from the model
    score = model.predict(normalized_keypoints).flatten()[0]

    # Clip the predicted score to ensure it's between 1 and 5
    score = np.clip(score, 1, 5)

    # Round the score to the nearest 0.5
    rounded_score = round(score * 2) / 2
    frame_scores.append(rounded_score)

    # Step 4: Annotate the frame with the score
    cv2.putText(frame, f"Score: {rounded_score:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show frame
    frame_count += 1
    cv2.imshow('Video', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Step 5: Aggregate Results
average_score = np.mean(frame_scores) if frame_scores else 0
print(f"Average Score for the Video: {average_score:.2f}")


Scaler saved at /Users/alessiacolumban/Desktop/TeamProject-GradindSysAthletes/Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl
Epoch 1/100


/opt/anaconda3/envs/athletes/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 4.9398 - mae: 1.8413 - val_loss: 8.6938 - val_mae: 2.7560
Epoch 2/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2882 - mae: 0.9416 - val_loss: 8.3691 - val_mae: 2.7000
Epoch 3/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1643 - mae: 0.8801 - val_loss: 8.9547 - val_mae: 2.8124
Epoch 4/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0189 - mae: 0.8212 - val_loss: 8.2019 - val_mae: 2.6782
Epoch 5/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9625 - mae: 0.7865 - val_loss: 7.4538 - val_mae: 2.5471
Epoch 6/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8904 - mae: 0.7612 - val_loss: 6.4688 - val_mae: 2.3669
Epoch 7/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8807 - mae: 0.7434 - val_loss: 6.7708 - val_mae: 2.4201
Epoch 8/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8975 - mae: 0.7625 - val_loss: 5.3756 - val_mae: 2.1379
Epoch 9/100
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7984 - mae: 0.7

R² Score: 0.6399

0: 384x640 1 person, 43.2ms
Speed: 0.9ms preprocess, 43.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 1 person, 57.6ms
Speed: 1.2ms preprocess, 57.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 1 person, 44.6ms
Speed: 0.8ms preprocess, 44.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 1 person, 43.8ms
Speed: 0.7ms preprocess, 43.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

0: 384x640 1 person, 46.6ms
Speed: 0.9ms preprocess, 46.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 1 person, 42.9ms
Speed: 0.8ms preprocess, 42.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

In [29]:
import cv2
import numpy as np
import json
import tensorflow as tf
from ultralytics import YOLO
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os

# Load JSON data for scoring model
file_path = "Athletes/KeypointDetection/JsonScore/Sprint_Start.json"
with open(file_path, 'r') as f:
    data = json.load(f)

# Define the expected number of keypoints (e.g., 17 keypoints, each with x, y, and visibility)
expected_keypoints = 17
expected_size = expected_keypoints * 3  # Each keypoint has x, y, visibility (3 values)

# Extract features (keypoints) and labels (scores)
features = []
scores = []

for segment, details in data['segments'].items():
    for annotation in details['annotations']:
        keypoints = annotation['keypoints']  # Flatten keypoints
        normalized_keypoints = []

        # Normalize using bbox dimensions
        bbox = annotation['bbox']
        
        # Ensure that keypoints contain a valid number of elements (x, y, and visibility for each keypoint)
        if len(keypoints) % 3 == 0:  # Check that each keypoint has x, y, and visibility
            for i in range(0, len(keypoints), 3):
                x = keypoints[i] / bbox[2]  # Normalize by width
                y = keypoints[i + 1] / bbox[3]  # Normalize by height
                visibility = keypoints[i + 2]  # Keep visibility as-is
                normalized_keypoints.extend([x, y, visibility])

            # Pad or trim the keypoints list to the expected size
            if len(normalized_keypoints) > expected_size:
                normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
            elif len(normalized_keypoints) < expected_size:
                normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

            features.append(normalized_keypoints)
            scores.append(details['score'])
        else:
            print(f"Skipping annotation with invalid keypoint length: {len(keypoints)}")

# Convert to NumPy arrays (after ensuring consistency)
X = np.array(features, dtype=np.float32)
y = np.array(scores, dtype=np.float32)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features using MinMaxScaler and save the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
scaler_path = "Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl"  # Correct path for saving the scaler
joblib.dump(scaler, scaler_path)  # Save the scaler
print(f"Scaler saved at {scaler_path}")

# Define a more complex model with added LSTM layers for sequential data
# def create_model(input_dim):
#     model = tf.keras.Sequential([
#         # Reshape the input to be 3D for LSTM
#         tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),  # Reshape to (1, features)
        
#         # LSTM layer for sequential data (even though you have one timestep)
#         tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
#         tf.keras.layers.LSTM(64, activation='relu'),
        
#         # Dense layers for feature learning
#         tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.2),
        
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
#     ])
#     model.compile(optimizer='adam', loss='mse', metrics=['mae'])
#     return model
def create_model(input_dim):
    model = tf.keras.Sequential([
        # Reshape input for sequential data
        tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),

        # LSTM layers
        tf.keras.layers.LSTM(128, activation='tanh', return_sequences=True, recurrent_dropout=0.2),
        tf.keras.layers.LSTM(64, activation='tanh', recurrent_dropout=0.2),
        tf.keras.layers.Dropout(0.2),

        # Dense layers with Batch Normalization and Dropout
        tf.keras.layers.Dense(128),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(64, activation='relu'),

        # Output layer
        tf.keras.layers.Dense(1, activation='linear')  # For regression
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])
    return model



# Initialize and train the model with early stopping
model = create_model(X_train.shape[1])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# Calculate R² score
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2:.4f}")

# Save the model
model.save("Sprint_Start.h5")

# Load YOLO model for pose estimation
pose_model = YOLO("yolov8n-pose.pt")

# Path to the video
video_path = "Athletes/Testing/Testing/SegmentedVideosOriginal/segment_000185.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
frame_scores = []

# Load the saved scaler for normalization of keypoints
if os.path.exists(scaler_path):
    scaler_scoring = joblib.load(scaler_path)
else:
    # If the scaler doesn't exist, fit a new one (this case should not happen if you have already saved it)
    scaler_scoring = MinMaxScaler(feature_range=(0, 1))
    scaler_scoring.fit(X)  # Use `X` as the keypoints data
    joblib.dump(scaler_scoring, scaler_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Extract Keypoints
    results = pose_model(frame)
    if len(results) == 0 or results[0].keypoints is None:
        continue

    keypoints = results[0].keypoints.xy.cpu().numpy().flatten()  # x, y coordinates
    if len(keypoints) == 0:
        continue  # Skip frames without valid keypoints

    # Step 2: Normalize keypoints using bounding box dimensions
    bbox = results[0].boxes.xyxy.cpu().numpy()[0]  # x_min, y_min, x_max, y_max
    if bbox is None or len(bbox) != 4:
        continue  # Skip frames without valid bbox

    normalized_keypoints = []
    for i in range(0, len(keypoints), 2):  # Assuming keypoints are in x, y pairs
        x = keypoints[i] / bbox[2]  # Normalize by width
        y = keypoints[i + 1] / bbox[3]  # Normalize by height
        normalized_keypoints.extend([x, y])

    # Pad or trim the normalized keypoints to match the expected feature size (51 values)
    if len(normalized_keypoints) > expected_size:
        normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
    elif len(normalized_keypoints) < expected_size:
        normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

    # Step 3: Predict Score using the trained scoring model
    normalized_keypoints = np.array([normalized_keypoints], dtype=np.float32)

    # Normalize the features using the saved scaler
    normalized_keypoints = scaler_scoring.transform(normalized_keypoints)

    # Predict score from the model
    score = model.predict(normalized_keypoints).flatten()[0]

    # Clip the predicted score to ensure it's between 1 and 5
    score = np.clip(score, 1, 5)

    # Round the score to the nearest 0.5
    rounded_score = round(score * 2) / 2
    frame_scores.append(rounded_score)

    # Step 4: Annotate the frame with the score
    cv2.putText(frame, f"Score: {rounded_score:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show frame
    frame_count += 1
    cv2.imshow('Video', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Step 5: Aggregate Results
average_score = np.mean(frame_scores) if frame_scores else 0
print(f"Average Score for the Video: {average_score:.2f}")


Scaler saved at /Users/alessiacolumban/Desktop/TeamProject-GradindSysAthletes/Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl
Epoch 1/100


/opt/anaconda3/envs/athletes/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 7.6616 - mae: 2.2805 - val_loss: 11.9953 - val_mae: 3.4003
Epoch 2/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6579 - mae: 0.6473 - val_loss: 12.3348 - val_mae: 3.4515
Epoch 3/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5735 - mae: 0.6039 - val_loss: 11.6283 - val_mae: 3.3513
Epoch 4/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5033 - mae: 0.5650 - val_loss: 10.5931 - val_mae: 3.1988
Epoch 5/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4783 - mae: 0.5420 - val_loss: 8.5048 - val_mae: 2.8512
Epoch 6/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4296 - mae: 0.5147 - val_loss: 6.8319 - val_mae: 2.5581
Epoch 7/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4408 - mae: 0.5261 - val_loss: 5.9025 - val_mae: 2.3647
Epoch 8/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4104 - mae: 0.5108 - val_loss: 3.4093 - val_mae: 1.7761
Epoch 9/100
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4007 - mae:

R² Score: 0.5111

0: 384x640 4 persons, 57.8ms
Speed: 1.5ms preprocess, 57.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 4 persons, 58.3ms
Speed: 2.8ms preprocess, 58.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

0: 384x640 5 persons, 54.4ms
Speed: 0.9ms preprocess, 54.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 5 persons, 58.9ms
Speed: 1.0ms preprocess, 58.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

0: 384x640 5 persons, 61.2ms
Speed: 0.7ms preprocess, 61.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

0: 384x640 4 persons, 110.2ms
Speed: 0.9ms preprocess, 110.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [32]:
import cv2
import numpy as np
import json
import tensorflow as tf
from ultralytics import YOLO
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os

# Load JSON data for scoring model
file_path = "Athletes/KeypointDetection/JsonScore/Sprint.json"
with open(file_path, 'r') as f:
    data = json.load(f)

# Define the expected number of keypoints (e.g., 17 keypoints, each with x, y, and visibility)
expected_keypoints = 17
expected_size = expected_keypoints * 3  # Each keypoint has x, y, visibility (3 values)

# Extract features (keypoints) and labels (scores)
features = []
scores = []

for segment, details in data['segments'].items():
    for annotation in details['annotations']:
        keypoints = annotation['keypoints']  # Flatten keypoints
        normalized_keypoints = []

        # Normalize using bbox dimensions
        bbox = annotation['bbox']
        
        # Ensure that keypoints contain a valid number of elements (x, y, and visibility for each keypoint)
        if len(keypoints) % 3 == 0:  # Check that each keypoint has x, y, and visibility
            for i in range(0, len(keypoints), 3):
                x = keypoints[i] / bbox[2]  # Normalize by width
                y = keypoints[i + 1] / bbox[3]  # Normalize by height
                visibility = keypoints[i + 2]  # Keep visibility as-is
                normalized_keypoints.extend([x, y, visibility])

            # Pad or trim the keypoints list to the expected size
            if len(normalized_keypoints) > expected_size:
                normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
            elif len(normalized_keypoints) < expected_size:
                normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

            features.append(normalized_keypoints)
            scores.append(details['score'])
        else:
            print(f"Skipping annotation with invalid keypoint length: {len(keypoints)}")

# Convert to NumPy arrays (after ensuring consistency)
X = np.array(features, dtype=np.float32)
y = np.array(scores, dtype=np.float32)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features using MinMaxScaler and save the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
scaler_path = "Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl"  # Correct path for saving the scaler
joblib.dump(scaler, scaler_path)  # Save the scaler
print(f"Scaler saved at {scaler_path}")

# Define a more complex model with added LSTM layers for sequential data
# def create_model(input_dim):
#     model = tf.keras.Sequential([
#         # Reshape the input to be 3D for LSTM
#         tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),  # Reshape to (1, features)
        
#         # LSTM layer for sequential data (even though you have one timestep)
#         tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
#         tf.keras.layers.LSTM(64, activation='relu'),
        
#         # Dense layers for feature learning
#         tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.3),
        
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
#     ])
#     model.compile(optimizer='adam', loss='mse', metrics=['mae'])
#     return model
def create_model(input_dim):
    model = tf.keras.Sequential([
        # Reshape input for sequential data
        tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),

        # LSTM layers
        tf.keras.layers.LSTM(128, activation='tanh', return_sequences=True, recurrent_dropout=0.2),
        tf.keras.layers.LSTM(64, activation='tanh', recurrent_dropout=0.2),
        tf.keras.layers.Dropout(0.2),

        # Dense layers with Batch Normalization and Dropout
        tf.keras.layers.Dense(128),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Dense(64, activation='relu'),

        # Output layer
        tf.keras.layers.Dense(1, activation='linear')  # For regression
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])
    return model



# Initialize and train the model with early stopping
model = create_model(X_train.shape[1])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# Calculate R² score
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2:.4f}")

# Save the model
model.save("Sprint.h5")

# Load YOLO model for pose estimation
pose_model = YOLO("yolov8n-pose.pt")

# Path to the video
video_path = "Athletes/KeypointDetection/exercises/sprint/segment_000729.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
frame_scores = []

# Load the saved scaler for normalization of keypoints
if os.path.exists(scaler_path):
    scaler_scoring = joblib.load(scaler_path)
else:
    # If the scaler doesn't exist, fit a new one (this case should not happen if you have already saved it)
    scaler_scoring = MinMaxScaler(feature_range=(0, 1))
    scaler_scoring.fit(X)  # Use `X` as the keypoints data
    joblib.dump(scaler_scoring, scaler_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Extract Keypoints
    results = pose_model(frame)
    if len(results) == 0 or results[0].keypoints is None:
        continue

    keypoints = results[0].keypoints.xy.cpu().numpy().flatten()  # x, y coordinates
    if len(keypoints) == 0:
        continue  # Skip frames without valid keypoints

    # Step 2: Normalize keypoints using bounding box dimensions
    bbox = results[0].boxes.xyxy.cpu().numpy()[0]  # x_min, y_min, x_max, y_max
    if bbox is None or len(bbox) != 4:
        continue  # Skip frames without valid bbox

    normalized_keypoints = []
    for i in range(0, len(keypoints), 2):  # Assuming keypoints are in x, y pairs
        x = keypoints[i] / bbox[2]  # Normalize by width
        y = keypoints[i + 1] / bbox[3]  # Normalize by height
        normalized_keypoints.extend([x, y])

    # Pad or trim the normalized keypoints to match the expected feature size (51 values)
    if len(normalized_keypoints) > expected_size:
        normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
    elif len(normalized_keypoints) < expected_size:
        normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

    # Step 3: Predict Score using the trained scoring model
    normalized_keypoints = np.array([normalized_keypoints], dtype=np.float32)

    # Normalize the features using the saved scaler
    normalized_keypoints = scaler_scoring.transform(normalized_keypoints)

    # Predict score from the model
    score = model.predict(normalized_keypoints).flatten()[0]

    # Clip the predicted score to ensure it's between 1 and 5
    score = np.clip(score, 1, 5)

    # Round the score to the nearest 0.5
    rounded_score = round(score * 2) / 2
    frame_scores.append(rounded_score)

    # Step 4: Annotate the frame with the score
    cv2.putText(frame, f"Score: {rounded_score:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show frame
    frame_count += 1
    cv2.imshow('Video', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Step 5: Aggregate Results
average_score = np.mean(frame_scores) if frame_scores else 0
print(f"Average Score for the Video: {average_score:.2f}")


Scaler saved at /Users/alessiacolumban/Desktop/TeamProject-GradindSysAthletes/Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl
Epoch 1/100


/opt/anaconda3/envs/athletes/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 8.9266 - mae: 2.5859 - val_loss: 11.6625 - val_mae: 3.2922
Epoch 2/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4810 - mae: 0.9729 - val_loss: 11.1664 - val_mae: 3.2177
Epoch 3/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1280 - mae: 0.8402 - val_loss: 10.4401 - val_mae: 3.1027
Epoch 4/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9048 - mae: 0.7681 - val_loss: 10.2175 - val_mae: 3.0703
Epoch 5/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0272 - mae: 0.8162 - val_loss: 10.0279 - val_mae: 3.0418
Epoch 6/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8758 - mae: 0.7411 - val_loss: 9.3531 - val_mae: 2.9375
Epoch 7/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7895 - mae: 0.6820 - val_loss: 8.8114 - val_mae: 2.8496
Epoch 8/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7239 - mae: 0.6657 - val_loss: 8.8338 - val_mae: 2.8552
Epoch 9/100
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6771 - ma

R² Score: 0.0149

0: 640x640 1 person, 101.5ms
Speed: 1.9ms preprocess, 101.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

0: 640x640 1 person, 98.0ms
Speed: 2.6ms preprocess, 98.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 640x640 1 person, 72.2ms
Speed: 1.9ms preprocess, 72.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 640x640 1 person, 73.0ms
Speed: 1.4ms preprocess, 73.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 640x640 1 person, 68.4ms
Speed: 1.4ms preprocess, 68.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 640x640 1 person, 70.6ms
Speed: 2.2ms preprocess, 70.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/st

In [43]:
import cv2
import numpy as np
import json
import tensorflow as tf
from ultralytics import YOLO
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import os

# Load JSON data for scoring model
file_path = "Athletes/KeypointDetection/JsonScore/Verspringen.json"
with open(file_path, 'r') as f:
    data = json.load(f)

# Define the expected number of keypoints (e.g., 17 keypoints, each with x, y, and visibility)
expected_keypoints = 17
expected_size = expected_keypoints * 3  # Each keypoint has x, y, visibility (3 values)

# Extract features (keypoints) and labels (scores)
features = []
scores = []

for segment, details in data['segments'].items():
    for annotation in details['annotations']:
        keypoints = annotation['keypoints']  # Flatten keypoints
        normalized_keypoints = []

        # Normalize using bbox dimensions
        bbox = annotation['bbox']
        
        # Ensure that keypoints contain a valid number of elements (x, y, and visibility for each keypoint)
        if len(keypoints) % 3 == 0:  # Check that each keypoint has x, y, and visibility
            for i in range(0, len(keypoints), 3):
                x = keypoints[i] / bbox[2]  # Normalize by width
                y = keypoints[i + 1] / bbox[3]  # Normalize by height
                visibility = keypoints[i + 2]  # Keep visibility as-is
                normalized_keypoints.extend([x, y, visibility])

            # Pad or trim the keypoints list to the expected size
            if len(normalized_keypoints) > expected_size:
                normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
            elif len(normalized_keypoints) < expected_size:
                normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

            features.append(normalized_keypoints)
            scores.append(details['score'])
        else:
            print(f"Skipping annotation with invalid keypoint length: {len(keypoints)}")

# Convert to NumPy arrays (after ensuring consistency)
X = np.array(features, dtype=np.float32)
y = np.array(scores, dtype=np.float32)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features using MinMaxScaler and save the scaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
scaler_path = "Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl"  # Correct path for saving the scaler
joblib.dump(scaler, scaler_path)  # Save the scaler
print(f"Scaler saved at {scaler_path}")

# Define a more complex model with added LSTM layers for sequential data
def create_model(input_dim):
    model = tf.keras.Sequential([
        # Reshape the input to be 3D for LSTM
        tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),  # Reshape to (1, features)
        
        # LSTM layer for sequential data (even though you have one timestep)
        tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
        tf.keras.layers.LSTM(64, activation='relu'),
        
        # Dense layers for feature learning
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# def create_model(input_dim):
#     model = tf.keras.Sequential([
#         # Reshape the input to be 3D for LSTM
#         tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),  # Reshape to (1, features)
        
#         # LSTM layers for sequential data
#         tf.keras.layers.LSTM(256, activation='relu', return_sequences=True),
#         tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
#         tf.keras.layers.LSTM(64, activation='relu'),
        
#         # Dense layers for feature learning
#         tf.keras.layers.Dense(256, activation='relu'),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.2),
        
#         tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.2),
        
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
#     ])
#     model.compile(optimizer='adam', loss='mse', metrics=['mae'])
#     return model

# Initialize and train the model with early stopping
model = create_model(X_train.shape[1])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# Calculate R² score
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2:.4f}")

# Save the model
model.save("Verspringen.h5")

# Load YOLO model for pose estimation
pose_model = YOLO("yolov8n-pose.pt")

# Path to the video
video_path = "Athletes/Testing/Testing/SegmentedVideosOriginal/segment_002121.mp4"
cap = cv2.VideoCapture(video_path)

frame_count = 0
frame_scores = []

# Load the saved scaler for normalization of keypoints
if os.path.exists(scaler_path):
    scaler_scoring = joblib.load(scaler_path)
else:
    # If the scaler doesn't exist, fit a new one (this case should not happen if you have already saved it)
    scaler_scoring = MinMaxScaler(feature_range=(0, 1))
    scaler_scoring.fit(X)  # Use `X` as the keypoints data
    joblib.dump(scaler_scoring, scaler_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Extract Keypoints
    results = pose_model(frame)
    if len(results) == 0 or results[0].keypoints is None:
        continue

    keypoints = results[0].keypoints.xy.cpu().numpy().flatten()  # x, y coordinates
    if len(keypoints) == 0:
        continue  # Skip frames without valid keypoints

    # Step 2: Normalize keypoints using bounding box dimensions
    bbox = results[0].boxes.xyxy.cpu().numpy()[0]  # x_min, y_min, x_max, y_max
    if bbox is None or len(bbox) != 4:
        continue  # Skip frames without valid bbox

    normalized_keypoints = []
    for i in range(0, len(keypoints), 2):  # Assuming keypoints are in x, y pairs
        x = keypoints[i] / bbox[2]  # Normalize by width
        y = keypoints[i + 1] / bbox[3]  # Normalize by height
        normalized_keypoints.extend([x, y])

    # Pad or trim the normalized keypoints to match the expected feature size (51 values)
    if len(normalized_keypoints) > expected_size:
        normalized_keypoints = normalized_keypoints[:expected_size]  # Trim if too long
    elif len(normalized_keypoints) < expected_size:
        normalized_keypoints += [0] * (expected_size - len(normalized_keypoints))  # Pad if too short

    # Step 3: Predict Score using the trained scoring model
    normalized_keypoints = np.array([normalized_keypoints], dtype=np.float32)

    # Normalize the features using the saved scaler
    normalized_keypoints = scaler_scoring.transform(normalized_keypoints)

    # Predict score from the model
    score = model.predict(normalized_keypoints).flatten()[0]

    # Clip the predicted score to ensure it's between 1 and 5
    score = np.clip(score, 1, 5)

    # Round the score to the nearest 0.5
    rounded_score = round(score * 2) / 2
    frame_scores.append(rounded_score)

    # Step 4: Annotate the frame with the score
    cv2.putText(frame, f"Score: {rounded_score:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show frame
    frame_count += 1
    cv2.imshow('Video', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Step 5: Aggregate Results
average_score = np.mean(frame_scores) if frame_scores else 0
print(f"Average Score for the Video: {average_score:.2f}")


Scaler saved at /Users/alessiacolumban/Desktop/TeamProject-GradindSysAthletes/Athletes/KeypointDetection/JsonKeypoints/Testing/scoring_scaler7.pkl
Epoch 1/100


/opt/anaconda3/envs/athletes/lib/python3.12/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 8.5516 - mae: 2.6283 - val_loss: 10.6605 - val_mae: 3.1678
Epoch 2/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2272 - mae: 0.8680 - val_loss: 8.8784 - val_mae: 2.8819
Epoch 3/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8328 - mae: 0.7277 - val_loss: 6.7602 - val_mae: 2.4932
Epoch 4/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8177 - mae: 0.7044 - val_loss: 5.2951 - val_mae: 2.1905
Epoch 5/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.6727 - mae: 0.6475 - val_loss: 3.9761 - val_mae: 1.8653
Epoch 6/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5956 - mae: 0.5954 - val_loss: 2.6691 - val_mae: 1.4890
Epoch 7/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8100 - mae: 0.7010 - val_loss: 2.1724 - val_mae: 1.2809
Epoch 8/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5924 - mae: 0.5932 - val_loss: 1.6927 - val_mae: 1.0923
Epoch 9/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5720 - mae: 0.

R² Score: 0.4298

0: 384x640 1 person, 50.1ms
Speed: 1.8ms preprocess, 50.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

0: 384x640 1 person, 39.6ms
Speed: 0.8ms preprocess, 39.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 1 person, 41.2ms
Speed: 0.7ms preprocess, 41.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step

0: 384x640 1 person, 39.1ms
Speed: 0.7ms preprocess, 39.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

0: 384x640 1 person, 43.1ms
Speed: 1.8ms preprocess, 43.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

0: 384x640 1 person, 69.9ms
Speed: 1.1ms preprocess, 69.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

In [ ]:
def create_model(input_dim):
    model = tf.keras.Sequential([
        # Reshape the input to be 3D for LSTM
        tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),  # Reshape to (1, features)
        
        # LSTM layers for sequential data
        tf.keras.layers.LSTM(256, activation='relu', return_sequences=True),
        tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
        tf.keras.layers.LSTM(64, activation='relu'),
        
        # Dense layers for feature learning
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
def create_model(input_dim):
    model = tf.keras.Sequential([
        # Reshape the input to be 3D for LSTM
        tf.keras.layers.Reshape((1, input_dim), input_shape=(input_dim,)),  # Reshape to (1, features)
        
        # LSTM layers for sequential data
        tf.keras.layers.LSTM(256, activation='relu', return_sequences=True),
        tf.keras.layers.LSTM(128, activation='relu', return_sequences=True),
        tf.keras.layers.LSTM(64, activation='relu'),
        
        # Dense layers for feature learning
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model